# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
!pip install scikit-learn==1.3.2
!pip install tqdm
!pip install seaborn
!pip install xgboost
!pip install scikit-optimize
!pip install scipy==1.13.1

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-pac

In [2]:
!pip install shap

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import IsolationForest

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint, uniform

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

### 데이터 읽어오기


In [5]:
ROOT_DIR = "data"
#ROOT_DIR = "/content/"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,1,OK,240.0,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,1,OK,240.0,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1,OK,1000.0,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1,OK,1000.0,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,1,OK,240.0,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,2023-09-14 15:50:00,1,OK,240.0,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,2024-04-10 14:20:00,1,OK,1000.0,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,2024-02-22 08:40:00,1,OK,240.0,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,2023-07-25 11:00:00,1,OK,1000.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [6]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,2023-09-15 13:20:00,1,OK,1000.0,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,2024-02-06 16:50:00,1,OK,1000.0,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,2023-07-14 11:30:00,1,OK,240.0,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,2023-11-03 08:00:00,1,OK,1000.0,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,2023-12-23 14:00:00,1,OK,240.0,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,2023-10-18 11:10:00,1,OK,1000.0,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,2024-01-02 15:00:00,1,OK,1000.0,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,2023-11-15 09:10:00,1,OK,240.0,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,2023-05-18 14:20:00,1,OK,240.0,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


In [7]:
hand_data = pd.read_excel(os.path.join(ROOT_DIR, "hand_data.xlsx"))
hand_data

,day,start_time,end_time,Dam Thickness - 1time / day,Fill thickness - 1time / day,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time,Line Temp. - Every break time,Line humidity - Every break time,OCR gap - Line#1 stage1,OCR gap - Line#1 stage2,OCR gap - Line#1 stage3,OCR gap - Line #2 stage1,OCR gap - Line #2 stage2,OCR gap - Line #2 stage3
0,240319,08:00:00,10:30:00,454.0,193.0,1728.0,2919.0,23.4,56.6,1.2,1.000,1.200,1.2,0.900,1.4
1,240319,10:45:00,12:00:00,454.0,193.0,1754.0,2962.0,24.0,50.9,1.2,1.000,1.200,1.2,0.900,1.4
2,240319,13:00:00,15:00:00,454.0,193.0,1708.0,2887.0,24.5,49.6,1.2,1.000,1.200,1.2,0.900,1.4
3,240319,15:15:00,18:00:00,454.0,193.0,1751.0,2946.0,24.7,53.4,1.2,1.000,1.200,1.2,1.200,1.4
4,240319,18:30:00,20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,240428,08:00:00,10:30:00,478.0,232.0,1642.0,2952.0,24.6,53.0,1.0,1.232,1.466,1.2,1.167,1.5
101,240428,10:45:00,12:00:00,478.0,232.0,1679.0,2942.0,24.8,52.7,1.0,1.232,1.466,1.2,1.167,1.5
102,240428,13:00:00,15:00:00,478.0,232.0,1552.0,2941.0,24.6,55.5,1.0,1.232,1.466,1.2,1.167,1.5
103,240428,15:15:00,18:00:00,478.0,232.0,1552.0,2934.0,24.3,54.6,1.0,1.232,1.466,1.2,1.167,1.5


In [8]:
pd.reset_option('display.max_rows', None)
train_data[['Collect Date_Dam','Collect Date_AutoClave','Collect Date_Fill1','Collect Date_Fill2']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Collect Date_Dam        40506 non-null  object
 1   Collect Date_AutoClave  40506 non-null  object
 2   Collect Date_Fill1      40506 non-null  object
 3   Collect Date_Fill2      40506 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [9]:
train_data['Collect Date_Dam'] = pd.to_datetime(train_data['Collect Date_Dam'])
train_data['Collect Date_AutoClave'] = pd.to_datetime(train_data['Collect Date_AutoClave'])
train_data['Collect Date_Fill1'] = pd.to_datetime(train_data['Collect Date_Fill1'])
train_data['Collect Date_Fill2'] = pd.to_datetime(train_data['Collect Date_Fill2'])

# 소요시간 계산
train_data['Fill1-Dam'] = train_data['Collect Date_Fill1'] - train_data['Collect Date_Dam']
train_data['Fill2-Fill1'] = train_data['Collect Date_Fill2'] - train_data['Collect Date_Fill1']
train_data['AutoClave-Fill2'] = train_data['Collect Date_AutoClave'] - train_data['Collect Date_Fill2']
# 소요시간을 분 단위로 변환
train_data['Fill1-Dam.min'] = train_data['Fill1-Dam'].dt.total_seconds() / 60
train_data['Fill2-Fill1.min'] = train_data['Fill2-Fill1'].dt.total_seconds() / 60
train_data['AutoClave-Fill2.min'] = train_data['AutoClave-Fill2'].dt.total_seconds() / 60


# 결과 확인
train_data[['Collect Date_Dam','Collect Date_AutoClave','Collect Date_Fill1','Collect Date_Fill2','Fill1-Dam.min','Fill2-Fill1.min','AutoClave-Fill2.min']]

,Collect Date_Dam,Collect Date_AutoClave,Collect Date_Fill1,Collect Date_Fill2,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,2024-04-25 11:10:00,2024-04-25 11:50:00,2024-04-25 11:20:00,2024-04-25 11:20:00,10.0,0.0,30.0
1,2023-09-19 14:30:00,2023-09-19 15:00:00,2023-09-19 14:30:00,2023-09-19 14:30:00,0.0,0.0,30.0
2,2024-03-05 09:30:00,2024-03-05 10:10:00,2024-03-05 09:30:00,2024-03-05 09:30:00,0.0,0.0,40.0
3,2023-09-25 15:40:00,2023-09-25 16:20:00,2023-09-25 15:40:00,2023-09-25 15:40:00,0.0,0.0,40.0
4,2023-06-27 13:20:00,2023-06-27 14:00:00,2023-06-27 13:20:00,2023-06-27 13:20:00,0.0,0.0,40.0
...,...,...,...,...,...,...,...
40501,2023-09-14 15:50:00,2023-09-14 16:30:00,2023-09-14 15:50:00,2023-09-14 15:50:00,0.0,0.0,40.0
40502,2024-04-10 14:20:00,2024-04-10 15:00:00,2024-04-10 14:20:00,2024-04-10 14:20:00,0.0,0.0,40.0
40503,2024-02-22 08:40:00,2024-02-22 09:20:00,2024-02-22 08:50:00,2024-02-22 08:50:00,10.0,0.0,30.0
40504,2023-07-25 11:00:00,2023-07-25 11:40:00,2023-07-25 11:00:00,2023-07-25 11:00:00,0.0,0.0,40.0


In [10]:
pd.reset_option('display.max_rows', None)

In [11]:
train_data.drop(columns=['Fill1-Dam','Fill2-Fill1','AutoClave-Fill2'], inplace=True)

In [12]:
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,1,OK,240.0,NaN,...,127,NaN,NaN,1,NaN,NaN,Normal,10.0,0.0,30.0
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,30.0
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1,OK,1000.0,NaN,...,73,NaN,NaN,1,NaN,NaN,Normal,0.0,0.0,40.0
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1,OK,1000.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,2023-09-14 15:50:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,2024-04-10 14:20:00,1,OK,1000.0,NaN,...,197,NaN,NaN,1,NaN,NaN,Normal,0.0,0.0,40.0
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,2024-02-22 08:40:00,1,OK,240.0,NaN,...,27,NaN,NaN,1,NaN,NaN,Normal,10.0,0.0,30.0
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,2023-07-25 11:00:00,1,OK,1000.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0


In [13]:
train_data[['Collect Date_Dam','Collect Date_AutoClave','Collect Date_Fill1','Collect Date_Fill2']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Collect Date_Dam        40506 non-null  datetime64[ns]
 1   Collect Date_AutoClave  40506 non-null  datetime64[ns]
 2   Collect Date_Fill1      40506 non-null  datetime64[ns]
 3   Collect Date_Fill2      40506 non-null  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 1.2 MB


In [14]:
train_data.drop(columns=['Collect Date_AutoClave','Collect Date_Fill1','Collect Date_Fill2'], inplace=True)

In [15]:
train_data['Collect Date_Dam'] = train_data['Collect Date_Dam'].astype(object)

In [16]:
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,1,OK,240.0,NaN,...,127,NaN,NaN,1,NaN,NaN,Normal,10.0,0.0,30.0
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,30.0
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1,OK,1000.0,NaN,...,73,NaN,NaN,1,NaN,NaN,Normal,0.0,0.0,40.0
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1,OK,1000.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,2023-09-14 15:50:00,1,OK,240.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,2024-04-10 14:20:00,1,OK,1000.0,NaN,...,197,NaN,NaN,1,NaN,NaN,Normal,0.0,0.0,40.0
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,2024-02-22 08:40:00,1,OK,240.0,NaN,...,27,NaN,NaN,1,NaN,NaN,Normal,10.0,0.0,30.0
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,2023-07-25 11:00:00,1,OK,1000.0,NaN,...,1,NaN,NaN,0,NaN,NaN,Normal,0.0,0.0,40.0


In [17]:
test_data['Collect Date_Dam'] = pd.to_datetime(test_data['Collect Date_Dam'])
test_data['Collect Date_AutoClave'] = pd.to_datetime(test_data['Collect Date_AutoClave'])
test_data['Collect Date_Fill1'] = pd.to_datetime(test_data['Collect Date_Fill1'])
test_data['Collect Date_Fill2'] = pd.to_datetime(test_data['Collect Date_Fill2'])

# 소요시간 계산
test_data['Fill1-Dam'] = test_data['Collect Date_Fill1'] - test_data['Collect Date_Dam']
test_data['Fill2-Fill1'] = test_data['Collect Date_Fill2'] - test_data['Collect Date_Fill1']
test_data['AutoClave-Fill2'] = test_data['Collect Date_AutoClave'] - test_data['Collect Date_Fill2']
# 소요시간을 분 단위로 변환
test_data['Fill1-Dam.min'] = test_data['Fill1-Dam'].dt.total_seconds() / 60
test_data['Fill2-Fill1.min'] = test_data['Fill2-Fill1'].dt.total_seconds() / 60
test_data['AutoClave-Fill2.min'] = test_data['AutoClave-Fill2'].dt.total_seconds() / 60


# 결과 확인
test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,...,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Fill1-Dam,Fill2-Fill1,AutoClave-Fill2,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,2023-09-15 13:20:00,1,OK,1000.0,...,0,NaN,NaN,NaN,0 days 00:10:00,0 days 00:00:00,0 days 00:30:00,10.0,0.0,30.0
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,2024-02-06 16:50:00,1,OK,1000.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:40:00,0.0,0.0,40.0
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,2023-07-14 11:30:00,1,OK,240.0,...,0,NaN,NaN,NaN,0 days 00:10:00,0 days 00:00:00,0 days 00:30:00,10.0,0.0,30.0
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,2023-11-03 08:00:00,1,OK,1000.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:30:00,0.0,0.0,30.0
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,2023-12-23 14:00:00,1,OK,240.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:10:00,0 days 00:30:00,0.0,10.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,2023-10-18 11:10:00,1,OK,1000.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:40:00,0.0,0.0,40.0
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,2024-01-02 15:00:00,1,OK,1000.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:30:00,0.0,0.0,30.0
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,2023-11-15 09:10:00,1,OK,240.0,...,1,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:30:00,0.0,0.0,30.0
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,2023-05-18 14:20:00,1,OK,240.0,...,0,NaN,NaN,NaN,0 days 00:00:00,0 days 00:00:00,0 days 00:30:00,0.0,0.0,30.0


In [18]:
test_data.drop(columns=['Fill1-Dam','Fill2-Fill1','AutoClave-Fill2'], inplace=True)

In [19]:
test_data.drop(columns=['Collect Date_AutoClave','Collect Date_Fill1','Collect Date_Fill2'], inplace=True)

In [20]:
test_data['Collect Date_Dam'] = test_data['Collect Date_Dam'].astype(object)

In [21]:
test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,...,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,2023-09-15 13:20:00,1,OK,1000.0,...,1,NaN,NaN,0,NaN,NaN,NaN,10.0,0.0,30.0
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,2024-02-06 16:50:00,1,OK,1000.0,...,256,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,40.0
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,2023-07-14 11:30:00,1,OK,240.0,...,1,NaN,NaN,0,NaN,NaN,NaN,10.0,0.0,30.0
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,2023-11-03 08:00:00,1,OK,1000.0,...,0,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,30.0
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,2023-12-23 14:00:00,1,OK,240.0,...,215,NaN,NaN,1,NaN,NaN,NaN,0.0,10.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,2023-10-18 11:10:00,1,OK,1000.0,...,131,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,40.0
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,2024-01-02 15:00:00,1,OK,1000.0,...,279,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,30.0
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,2023-11-15 09:10:00,1,OK,240.0,...,66,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,30.0
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,2023-05-18 14:20:00,1,OK,240.0,...,1,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,30.0


## 📌 전처리 pt.1

### drop NA columns

In [22]:
def drop_missing_and_lot(df):
    """
    결측치가 절반 이상인 열과 'LOT ID - Dam' 열을 삭제하는 함수

    Args:
        df: 입력 데이터프레임

    Returns:
        처리된 데이터프레임
    """

    # 결측치가 절반 이상인 열 삭제
    drop_cols = []
    for column in df.columns:
        if (df[column].notnull().sum() // 2) < df[column].isnull().sum():
            drop_cols.append(column)
            #print(column)
    #print(drop_cols)
    df = df.drop(drop_cols, axis=1)

    # 'LOT ID - Dam' 열 삭제
    if 'LOT ID - Dam' in df.columns:
        df = df.drop("LOT ID - Dam", axis=1)

    return df

# 함수 적용
df_train = drop_missing_and_lot(train_data)  # 학습 데이터
df_test = drop_missing_and_lot(test_data)  # 테스트 데이터

In [23]:
df_train.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,...,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,2024-04-25 11:10:00,1,OK,240.0,2.5,...,50,114.612,19.9,7,127,1,Normal,10.0,0.0,30.0
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,2023-09-19 14:30:00,1,OK,240.0,2.5,...,85,19.600,7.0,185,1,0,Normal,0.0,0.0,30.0
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,2024-03-05 09:30:00,1,OK,1000.0,12.5,...,50,114.612,19.8,10,73,1,Normal,0.0,0.0,40.0
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,2023-09-25 15:40:00,1,OK,1000.0,12.5,...,85,19.900,12.0,268,1,0,Normal,0.0,0.0,40.0
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,2023-06-27 13:20:00,1,OK,240.0,2.5,...,85,19.700,8.0,121,1,0,Normal,0.0,0.0,40.0


In [24]:
df_test.head()

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,...,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,Fill1-Dam.min,Fill2-Fill1.min,AutoClave-Fill2.min
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,2023-09-15 13:20:00,1,OK,1000.0,...,50,85,19.8,13.0,195,1,0,10.0,0.0,30.0
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,2024-02-06 16:50:00,1,OK,1000.0,...,270,50,85.0,19.8,14,256,1,0.0,0.0,40.0
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,2023-07-14 11:30:00,1,OK,240.0,...,50,85,19.7,1.0,98,1,0,10.0,0.0,30.0
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,2023-11-03 08:00:00,1,OK,1000.0,...,270,50,85.0,20.0,14,0,1,0.0,0.0,30.0
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,2023-12-23 14:00:00,1,OK,240.0,...,270,50,85.0,19.8,1,215,1,0.0,10.0,30.0


In [25]:
# 각 데이터프레임의 열 이름 추출
train_columns = set(df_train.columns)
test_columns = set(df_test.columns)

# 차집합 구하기
diff_columns = test_columns.difference(train_columns)
diff_columns2 = train_columns.difference(test_columns)

# 결과 출력
print("test_data에만 있는 열:", diff_columns)
print("train_data에만 있는 열:", diff_columns2)

test_data에만 있는 열: {'Set ID'}
train_data에만 있는 열: {'target'}


In [26]:
# target
df_train['target'].value_counts()

target
Normal      38156
AbNormal     2350
Name: count, dtype: int64

In [27]:
from sklearn.impute import KNNImputer

def na_knn(df,columns):

  df_na = df[columns].replace("OK",np.nan)

  # KNN Imputer 사용
  imputer = KNNImputer(n_neighbors=3)
  df_imputed = pd.DataFrame(imputer.fit_transform(df_na), columns=df_na.columns)

  return df_imputed

columns = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
          'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam']
df_impute_dam =  na_knn(df_train,columns)['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']
df_impute_dam_test = na_knn(df_test,columns)['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']

columns = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
          'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2']
df_impute_fill2 =  na_knn(df_train,columns)['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']
df_impute_fill2_test = na_knn(df_test,columns)['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']

In [28]:
df_train['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = df_impute_dam
df_train['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = df_impute_fill2

df_test['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = df_impute_dam_test
df_test['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = df_impute_fill2_test

## 📌  전처리 pt.2

### class preprocess

In [29]:
class preprocess:

    def __init__(self, df):
        self.df = df

    def drop_features_from_df(self, drop_features):

        self.df = self.df.drop(drop_features, axis=1)
        return self.df

    def impute_na(self, col_names, seed=42):

        df_copy = self.df.copy()
        for col_name in col_names:

            df_copy[col_name] = df_copy[col_name].replace("OK", np.nan) #inplace=True)


            non_na_values = df_copy[col_name].dropna().values
            value_counts = pd.Series(non_na_values).value_counts(normalize=True)


            na_count = df_copy[col_name].isna().sum()


            np.random.seed(seed)
            imputed_values = np.random.choice(value_counts.index, na_count, p=value_counts.values)


            df_copy.loc[df_copy[col_name].isna(), col_name] = imputed_values

        self.df = df_copy
        return self.df

    def create_vector_columns(self, vector_columns_dict):

        for vector_name, columns in vector_columns_dict.items():
            self.df[vector_name] = self.df[columns].apply(lambda row: tuple(row), axis=1)
        return self.df

    def convert_to_category(self, columns):

        for col in columns:
            self.df[col] = self.df[col].astype('int').astype('category')
        return self.df

    def encode_columns(self, categorical_columns):

        self.df = pd.get_dummies(self.df, columns=categorical_columns, drop_first=False)
        return self.df

    def convert_head_normal_to_float(self):

        head_normal_cols = [col for col in self.df.columns
                            if col.startswith('HEAD NORMAL') and self.df[col].dtype == 'object']
        for col in head_normal_cols:
            try:
                self.df[col] = self.df[col].astype('float')
            except ValueError:
                print(f"Column '{col}' could not be converted to float. Check for non-numerical values.")

        return self.df


    def encode_categorical_features(self, rare_threshold=5, is_train=True):
        df_encoded = self.df.copy()
        categorical_cols = df_encoded.select_dtypes(include=['object','category']).columns


        for col in categorical_cols:
            if is_train and col == 'target':
                continue
            num_unique_values = df_encoded[col].nunique()
            if num_unique_values < rare_threshold:

                ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
                encoded_data = ohe.fit_transform(df_encoded[[col]])
                encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out([col]))
                df_encoded = pd.concat([df_encoded.drop(col, axis=1), encoded_df], axis=1)

            else:

                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df_encoded[col])
                df_encoded[col] = df_encoded[col].astype('category')
        self.df = df_encoded
        return self.df

    def bool_to_int(self):
        bool_cols = self.df.select_dtypes(include='bool').columns
        for col in bool_cols:
            self.df[col] = self.df[col].astype(int).astype('category')
        return self.df


    # target encoding
    def encode_target_column(self):

        self.df['target'] = self.df['target'].replace({'AbNormal': 1, 'Normal': 0}) #, inplace=True)
        self.df['target'] = self.df['target'].astype('category')
        return self.df

In [30]:
preprocessor = preprocess(df_train)
preprocessor_test = preprocess(df_test)

# 단일 값 열 제거
drop_features = ['Wip Line_Dam', 'Wip Line_AutoClave', 'Wip Line_Fill1', 'Wip Line_Fill2',
                 'Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2',
                 'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2',
                 'Insp. Seq No._AutoClave', 'Insp. Seq No._Fill1', 'Insp. Seq No._Fill2',
                 'Insp Judge Code_Dam', 'Insp Judge Code_AutoClave',
                 'Insp Judge Code_Fill1', 'Insp Judge Code_Fill2']

preprocessor.drop_features_from_df(drop_features)
preprocessor_test.drop_features_from_df(drop_features)

# 결측치 대체
'''
columns_to_impute = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
                     'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
                     'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']
'''
columns_to_impute = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']

preprocessor.impute_na(columns_to_impute)
preprocessor_test.impute_na(columns_to_impute)

# CURE 좌표 변수 - 범주화 & 기존 변수 삭제
vector_columns_dict_fill2 = {
    'CURE END POSITION VECTOR_Fill2': ['CURE END POSITION X Collect Result_Fill2', 'CURE END POSITION Z Collect Result_Fill2', 'CURE END POSITION Θ Collect Result_Fill2'],
    'CURE STANDBY POSITION VECTOR_Fill2': ['CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Z Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2'],
    'CURE START POSITION VECTOR_Fill2': ['CURE START POSITION X Collect Result_Fill2', 'CURE START POSITION Z Collect Result_Fill2', 'CURE START POSITION Θ Collect Result_Fill2']
}
vector_columns_dict_dam = {
    'CURE END POSITION VECTOR_Dam': ['CURE END POSITION X Collect Result_Dam', 'CURE END POSITION Z Collect Result_Dam', 'CURE END POSITION Θ Collect Result_Dam'],
    'CURE STANDBY POSITION VECTOR_Dam': ['CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam'],
    'CURE START POSITION VECTOR_Dam': ['CURE START POSITION X Collect Result_Dam', 'CURE START POSITION Z Collect Result_Dam', 'CURE START POSITION Θ Collect Result_Dam']
}

columns_to_drop_fill2 = vector_columns_dict_fill2['CURE END POSITION VECTOR_Fill2'] + vector_columns_dict_fill2['CURE STANDBY POSITION VECTOR_Fill2'] + vector_columns_dict_fill2['CURE START POSITION VECTOR_Fill2']
columns_to_drop_dam = vector_columns_dict_dam['CURE END POSITION VECTOR_Dam'] + vector_columns_dict_dam['CURE STANDBY POSITION VECTOR_Dam'] + vector_columns_dict_dam['CURE START POSITION VECTOR_Dam']

vector_columns_dict = {**vector_columns_dict_fill2, **vector_columns_dict_dam}
columns_to_drop = columns_to_drop_fill2 + columns_to_drop_dam
categorical_columns = ['CURE END POSITION VECTOR_Fill2', 'CURE STANDBY POSITION VECTOR_Fill2', 'CURE START POSITION VECTOR_Fill2',
                       'CURE END POSITION VECTOR_Dam', 'CURE STANDBY POSITION VECTOR_Dam', 'CURE START POSITION VECTOR_Dam']


preprocessor.create_vector_columns(vector_columns_dict)
preprocessor_test.create_vector_columns(vector_columns_dict)
preprocessor.drop_features_from_df(columns_to_drop)
preprocessor_test.drop_features_from_df(columns_to_drop)
preprocessor.encode_columns(categorical_columns) # 좌표 변수 one hot encoding w/ pd.dummies
preprocessor_test.encode_columns(categorical_columns)

# 단일값 컬럼 제거
constant_columns = [col for col in preprocessor.df.columns
                    if preprocessor.df[col].nunique() == 1]
preprocessor.drop_features_from_df(constant_columns)
preprocessor_test.drop_features_from_df(constant_columns)

preprocessor.convert_head_normal_to_float() # b/c 결측치 대체시 category로 됐음.
preprocessor_test.convert_head_normal_to_float()
preprocessor.encode_categorical_features(rare_threshold=5, is_train=True)
preprocessor_test.encode_categorical_features(rare_threshold=5, is_train=False)

# head column (상관관계 높은거 제거)
drop_head_columns = [
    'HEAD Standby Position X Collect Result_Fill1',
    'HEAD Standby Position Y Collect Result_Fill1',
    'HEAD Standby Position Z Collect Result_Fill1',
    'HEAD Standby Position X Collect Result_Fill2',
    'HEAD Standby Position Y Collect Result_Fill2',
    'HEAD Standby Position Z Collect Result_Fill2',
    'Head Clean Position X Collect Result_Fill1',
    'Head Clean Position Y Collect Result_Fill1',
    'Head Clean Position Z Collect Result_Fill1',
    'Head Clean Position X Collect Result_Fill2',
    'Head Clean Position Y Collect Result_Fill2',
    'Head Clean Position Z Collect Result_Fill2',
    'Head Purge Position X Collect Result_Fill1',
    'Head Purge Position Y Collect Result_Fill1',
    'Head Purge Position Z Collect Result_Fill1',
    'Head Purge Position X Collect Result_Fill2',
    'Head Purge Position Y Collect Result_Fill2',
    'Head Purge Position Z Collect Result_Fill2',
]
preprocessor.drop_features_from_df(drop_head_columns)
preprocessor_test.drop_features_from_df(drop_head_columns)

# bool을 int(category)로 변경
preprocessor.bool_to_int()
preprocessor_test.bool_to_int()

# some columns to category type
categorical_columns2 = ['Equipment_Dam_Dam dispenser #2','Chamber Temp. Judge Value_AutoClave_OK',
                        'Equipment_Fill1_Fill1 dispenser #2','Equipment_Fill2_Fill2 dispenser #2']
preprocessor.convert_to_category(categorical_columns2)
preprocessor_test.convert_to_category(categorical_columns2)



# target encoding
preprocessor.encode_target_column()

# final
df_train_preprocess = preprocessor.df
df_test_preprocess = preprocessor_test.df

In [31]:
pd.set_option('display.max_rows', None)
pd.DataFrame([df_train_preprocess.dtypes,
              df_train_preprocess.nunique()]).T

,0,1
Model.Suffix_Dam,category,7
Workorder_Dam,category,663
Collect Date_Dam,category,8889
CURE SPEED Collect Result_Dam,int64,5
DISCHARGED SPEED OF RESIN Collect Result_Dam,int64,3
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,float64,19
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,float64,29
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,float64,20
Dispense Volume(Stage1) Collect Result_Dam,float64,23
Dispense Volume(Stage2) Collect Result_Dam,float64,33


In [32]:
df_train_preprocess.head()

,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,"CURE START POSITION VECTOR_Fill2_(1020, 32, 90)","CURE START POSITION VECTOR_Fill2_(1020, 33, 90)","CURE END POSITION VECTOR_Dam_(240.0, 2.5, -90.0)","CURE END POSITION VECTOR_Dam_(1000.0, 12.5, 90.0)","CURE START POSITION VECTOR_Dam_(280.0, 33.5, 90.0)","CURE START POSITION VECTOR_Dam_(1030.0, 33.5, -90.0)",Equipment_Dam_Dam dispenser #2,Chamber Temp. Judge Value_AutoClave_OK,Equipment_Fill1_Fill1 dispenser #2,Equipment_Fill2_Fill2 dispenser #2
0,3,657,8816,100,16,14.9,8.4,14.7,1.04,0.58,...,0,1,1,0,0,1,0,1,0,0
1,3,283,4042,70,10,21.3,4.9,21.3,1.49,0.34,...,0,1,1,0,0,1,0,1,0,0
2,0,589,7893,85,16,14.7,8.5,14.7,1.61,0.93,...,0,1,0,1,1,0,1,1,1,1
3,0,251,4230,70,10,21.3,8.4,21.3,1.49,0.58,...,0,1,0,1,1,0,1,1,1,1
4,0,142,2057,70,10,9.7,4.9,9.6,0.67,0.34,...,0,1,1,0,0,1,0,0,0,0


In [33]:
pd.reset_option('display.max_rows')

In [34]:
# 음수값 개수 확인
for column in df_train_preprocess.columns:
    if df_train_preprocess[column].dtype.name != 'category':  # Check if column is not categorical
        negative_count = (df_train_preprocess[column] < 0).sum()
        print(f"Column '{column}' has {negative_count} negative values.")
    else:
        print(f"Column '{column}' is categorical, skipping negative value check.")

Column 'Model.Suffix_Dam' is categorical, skipping negative value check.
Column 'Workorder_Dam' is categorical, skipping negative value check.
Column 'Collect Date_Dam' is categorical, skipping negative value check.
Column 'CURE SPEED Collect Result_Dam' has 0 negative values.
Column 'DISCHARGED SPEED OF RESIN Collect Result_Dam' has 0 negative values.
Column 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam' has 0 negative values.
Column 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam' has 0 negative values.
Column 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam' has 0 negative values.
Column 'Dispense Volume(Stage1) Collect Result_Dam' has 0 negative values.
Column 'Dispense Volume(Stage2) Collect Result_Dam' has 0 negative values.
Column 'Dispense Volume(Stage3) Collect Result_Dam' has 0 negative values.
Column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam' has 0 negative values.
Column 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam' has 0 negativ

### feature engineering (no scaling)

In [35]:
class FeatureEngineering:

    def __init__(self, df):
        self.df = df

    def stage123_std(self):
        """
        Calculates the standard deviation

        Args:
          df: Pandas DataFrame containing the columns.

        Returns:
          float: The calculated standard deviation.
        """
        # Identify column groups based on prefixes
        column_groups = {'discharged_resin_time_dam': ['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
                                             'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
                                             'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'],
                         'discharged_resin_time_fill1':['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
                                                        'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
                                                        'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1'],
                         'dispensed_volume_dam': ['Dispense Volume(Stage1) Collect Result_Dam',
                                             'Dispense Volume(Stage2) Collect Result_Dam',
                                             'Dispense Volume(Stage3) Collect Result_Dam'],
                         'dispensed_volume_fill1': ['Dispense Volume(Stage1) Collect Result_Fill1',
                                             'Dispense Volume(Stage2) Collect Result_Fill1',
                                             'Dispense Volume(Stage3) Collect Result_Fill1'],
                         'head_normal_x_dam': ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'],
                         'head_normal_y_dam': ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'],
                         'head_normal_z_dam': ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
                                             'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
                                               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'],
                         'head_normal_x_fill1': ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'],
                         'head_normal_y_fill1': ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'],
                         'head_normal_z_fill1': ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
                                             'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'],
                         'head_normal_x_fill2': ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'],
                         'head_normal_y_fill2': ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'],
                         'head_normal_z_fill2': ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2',
                                             'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'],
                         'machine_tact_time': ['Machine Tact time Collect Result_Dam',
                                             'Machine Tact time Collect Result_Fill1',
                                             'Machine Tact time Collect Result_Fill2'],
                         'production_qty': ['Production Qty Collect Result_Dam',
                                             'Production Qty Collect Result_Fill1',
                                             'Production Qty Collect Result_Fill2'],
                         'pressure_autoclave': ['1st Pressure Collect Result_AutoClave',
                                                '2nd Pressure Collect Result_AutoClave',
                                              '3rd Pressure Collect Result_AutoClave'],
                         'pressure_time_autoclave': ['1st Pressure 1st Pressure Unit Time_AutoClave',
                                                '2nd Pressure Unit Time_AutoClave',
                                                '3rd Pressure Unit Time_AutoClave']
                         }

        for group_name, group_columns in column_groups.items():
            std_value = self.df[group_columns].std(axis=1)
            self.df[f'{group_name}_std'] = std_value

        return self.df

    def sum_columns(self):
        thickness_col = ['THICKNESS 2 Collect Result_Dam',
                          'THICKNESS 3 Collect Result_Dam']
        new_col_name = 'THICKNESS 2+3 Collect Result_Dam'
        self.df[new_col_name] = self.df[thickness_col].sum(axis=1)
        #self.df = self.df.drop(thickness_col, axis=1)
        return self.df

    def mean_columns(self):
        col1_c = ['Stage1 Circle1 Distance Speed Collect Result_Dam',
        'Stage1 Circle2 Distance Speed Collect Result_Dam',
        'Stage1 Circle3 Distance Speed Collect Result_Dam',
        'Stage1 Circle4 Distance Speed Collect Result_Dam']
        col1_l = ['Stage1 Line1 Distance Speed Collect Result_Dam',
        'Stage1 Line2 Distance Speed Collect Result_Dam',
        'Stage1 Line3 Distance Speed Collect Result_Dam',
        'Stage1 Line4 Distance Speed Collect Result_Dam']
        col2_c = ['Stage2 Circle1 Distance Speed Collect Result_Dam',
        'Stage2 Circle2 Distance Speed Collect Result_Dam',
        'Stage2 Circle3 Distance Speed Collect Result_Dam',
        'Stage2 Circle4 Distance Speed Collect Result_Dam']
        col2_l = ['Stage2 Line1 Distance Speed Collect Result_Dam',
        'Stage2 Line2 Distance Speed Collect Result_Dam',
        'Stage2 Line3 Distance Speed Collect Result_Dam',
        'Stage2 Line4 Distance Speed Collect Result_Dam']
        col3_c = ['Stage3 Circle1 Distance Speed Collect Result_Dam',
        'Stage3 Circle2 Distance Speed Collect Result_Dam',
        'Stage3 Circle3 Distance Speed Collect Result_Dam',
        'Stage3 Circle4 Distance Speed Collect Result_Dam']
        col3_l = ['Stage3 Line1 Distance Speed Collect Result_Dam',
        'Stage3 Line2 Distance Speed Collect Result_Dam',
        'Stage3 Line3 Distance Speed Collect Result_Dam',
        'Stage3 Line4 Distance Speed Collect Result_Dam']
        column_groups = {'Stage1 Circle1~4 Distance Speed Collect Result_Dam': col1_c,
                         'Stage1 Line1~4 Distance Speed Collect Result_Dam':col1_l,
                         'Stage2 Circle1~4 Distance Speed Collect Result_Dam':col2_c,
                         'Stage2 Line1~4 Distance Speed Collect Result_Dam':col2_l,
                         'Stage3 Circle1~4 Distance Speed Collect Result_Dam':col3_c,
                         'Stage3 Line1~4 Distance Speed Collect Result_Dam':col3_l}
        for group_name, group_columns in column_groups.items():
            mean_val = self.df[group_columns].mean(axis=1)
            self.df[f'{group_name}'] = mean_val
            self.df = self.df.drop(group_columns, axis=1)
        return self.df


    def scaling(self):
        scaler = MinMaxScaler()
        numeric_cols = self.df.select_dtypes(include=['float64', 'int64']).columns
        self.df[numeric_cols] = scaler.fit_transform(self.df[numeric_cols])
        return self.df

    def feature_enginering(self):
        self.df = self.stage123_std()
        self.df = self.sum_columns()
        self.df = self.mean_columns()
        #self.df = self.scaling()
        return self.df

In [36]:
fe = FeatureEngineering(df_train_preprocess)
df_train2 = fe.feature_enginering()

fe_test = FeatureEngineering(df_test_preprocess)
df_test2 = fe_test.feature_enginering()

In [37]:
df_train2.head()

,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,production_qty_std,pressure_autoclave_std,pressure_time_autoclave_std,THICKNESS 2+3 Collect Result_Dam,Stage1 Circle1~4 Distance Speed Collect Result_Dam,Stage1 Line1~4 Distance Speed Collect Result_Dam,Stage2 Circle1~4 Distance Speed Collect Result_Dam,Stage2 Line1~4 Distance Speed Collect Result_Dam,Stage3 Circle1~4 Distance Speed Collect Result_Dam,Stage3 Line1~4 Distance Speed Collect Result_Dam
0,3,657,8816,100,16,14.9,8.4,14.7,1.04,0.58,...,0.0,0.106275,119.500349,0.00,4350.25,5750.0,5425.0,5300.0,5675.0,5800.0
1,3,283,4042,70,10,21.3,4.9,21.3,1.49,0.34,...,0.0,0.107965,69.282032,0.00,4000.00,4000.0,9000.0,9000.0,4000.0,4000.0
2,0,589,7893,85,16,14.7,8.5,14.7,1.61,0.93,...,0.0,0.103423,120.000000,-0.01,4350.25,5800.0,5425.0,5300.0,5675.0,5800.0
3,0,251,4230,70,10,21.3,8.4,21.3,1.49,0.58,...,0.0,0.115760,69.282032,0.00,4000.00,4000.0,5000.0,5000.0,4000.0,4000.0
4,0,142,2057,70,10,9.7,4.9,9.6,0.67,0.34,...,0.0,0.113161,69.282032,0.00,9000.00,9000.0,9000.0,9000.0,9000.0,9000.0


In [38]:
pd.set_option('display.max_rows', None)
pd.DataFrame([df_train2.dtypes,
              df_train2.nunique()]).T

,0,1
Model.Suffix_Dam,category,7
Workorder_Dam,category,663
Collect Date_Dam,category,8889
CURE SPEED Collect Result_Dam,int64,5
DISCHARGED SPEED OF RESIN Collect Result_Dam,int64,3
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,float64,19
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,float64,29
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,float64,20
Dispense Volume(Stage1) Collect Result_Dam,float64,23
Dispense Volume(Stage2) Collect Result_Dam,float64,33


## 📌 모델 학습


### train test split

In [39]:
df_train_final = df_train2.copy()
df_test_final = df_test2.copy()

In [40]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [41]:
df_train_final.head()

,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,production_qty_std,pressure_autoclave_std,pressure_time_autoclave_std,THICKNESS 2+3 Collect Result_Dam,Stage1 Circle1~4 Distance Speed Collect Result_Dam,Stage1 Line1~4 Distance Speed Collect Result_Dam,Stage2 Circle1~4 Distance Speed Collect Result_Dam,Stage2 Line1~4 Distance Speed Collect Result_Dam,Stage3 Circle1~4 Distance Speed Collect Result_Dam,Stage3 Line1~4 Distance Speed Collect Result_Dam
0,3,657,8816,100,16,14.9,8.4,14.7,1.04,0.58,...,0.0,0.106275,119.500349,0.00,4350.25,5750.0,5425.0,5300.0,5675.0,5800.0
1,3,283,4042,70,10,21.3,4.9,21.3,1.49,0.34,...,0.0,0.107965,69.282032,0.00,4000.00,4000.0,9000.0,9000.0,4000.0,4000.0
2,0,589,7893,85,16,14.7,8.5,14.7,1.61,0.93,...,0.0,0.103423,120.000000,-0.01,4350.25,5800.0,5425.0,5300.0,5675.0,5800.0
3,0,251,4230,70,10,21.3,8.4,21.3,1.49,0.58,...,0.0,0.115760,69.282032,0.00,4000.00,4000.0,5000.0,5000.0,4000.0,4000.0
4,0,142,2057,70,10,9.7,4.9,9.6,0.67,0.34,...,0.0,0.113161,69.282032,0.00,9000.00,9000.0,9000.0,9000.0,9000.0,9000.0


In [42]:
# Assuming 'target' is the name of your target variable column
X_final = df_train_final.drop(['target'], axis=1)
y_final = df_train_final['target']

In [43]:
# 양성 클래스 (AbNormal)와 음성 클래스 (Normal) 샘플 수 계산
positive_samples = y_final[y_final == 1].count()
negative_samples = y_final[y_final == 0].count()

# scale_pos_weight 계산
scale_pos_weight = negative_samples / positive_samples
scale_pos_weight

16.23659574468085

### 🌳 RandomForest

In [43]:
'''
rfmodel = RandomForestClassifier()

param_dist = {
    'n_estimators': randint(100, 500),  # 더 많은 트리를 탐색 (복잡도 증가)
    'max_depth': randint(5, 30),  # 더 깊은 트리를 탐색 (복잡도 증가)
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),  # 과적합 방지를 위해 잎 노드의 최소 샘플 수 조정
    'criterion': ['gini', 'entropy'],  # 'log_loss'는 불균형 데이터에서 불안정할 수 있음
    'class_weight': ['balanced_subsample'] # 클래스 가중치 옵션 추가
}

random_search = RandomizedSearchCV(
    estimator=rfmodel,
    param_distributions=param_dist,
    n_iter=50,  # 탐색할 하이퍼파라미터 조합 수
    cv=5, # 교차 검증 폴드 수
    scoring='f1',  # 평가 지표
    random_state=42,  # 랜덤 시드 설정
    n_jobs=1
)

random_search.fit(X_final, y_final)

print("Best hyperparameters found: ", random_search.best_params_)

best_model = random_search.best_estimator_
'''

Best hyperparameters found:  {'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 21, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 471}


{'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 21, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 471}

In [44]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


f1_scores = []
precision_scores = []
recall_scores = []

feature_importances = []


for train_index, val_index in skf.split(X_final, y_final):
    X_train, X_val = X_final.iloc[train_index], X_final.iloc[val_index]
    y_train, y_val = y_final.iloc[train_index], y_final.iloc[val_index]


    rfmodel = RandomForestClassifier(
        n_estimators=471,
        max_depth=21,
        criterion='entropy',
        random_state=42,
        min_samples_split=6,
        min_samples_leaf=6,
        class_weight='balanced_subsample'
    )
    rfmodel.fit(X_train, y_train)


    y_pred = rfmodel.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    f1_scores.append(f1)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    precision_scores.append(precision)
    recall_scores.append(recall)

    cm = confusion_matrix(y_val, y_pred)
    print('Confusion Matrix:')
    print(cm)

    print('f1 score:', f1)
    print('precision score:', precision)
    print('recall score:', recall)
    print()


    feature_importances.append(rfmodel.feature_importances_)


print("Average F1 Score:", np.mean(f1_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))


mean_feature_importances = np.mean(feature_importances, axis=0)


features = X_final.columns
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': mean_feature_importances
}).sort_values(by='Importance', ascending=False)


print(importance_df.head(10))

Confusion Matrix:
[[7499  133]
 [ 371   99]]
f1 score: 0.28205128205128205
precision score: 0.4267241379310345
recall score: 0.21063829787234042

Confusion Matrix:
[[7499  132]
 [ 381   89]]
f1 score: 0.2575976845151954
precision score: 0.40271493212669685
recall score: 0.18936170212765957

Confusion Matrix:
[[7526  105]
 [ 381   89]]
f1 score: 0.2680722891566265
precision score: 0.4587628865979381
recall score: 0.18936170212765957

Confusion Matrix:
[[7495  136]
 [ 391   79]]
f1 score: 0.23065693430656936
precision score: 0.3674418604651163
recall score: 0.16808510638297872

Confusion Matrix:
[[7523  108]
 [ 375   95]]
f1 score: 0.2823179791976226
precision score: 0.46798029556650245
recall score: 0.20212765957446807

Average F1 Score: 0.26413923384545923
Average Precision: 0.4247248225374577
Average Recall: 0.19191489361702127
                                   Feature  Importance
2                         Collect Date_Dam    0.054482
123                 pressure_autoclave_std    0.0

In [45]:
print(importance_df.head(30))

                                               Feature  Importance
2                                     Collect Date_Dam    0.054482
123                             pressure_autoclave_std    0.044912
1                                        Workorder_Dam    0.043424
86                                 AutoClave-Fill2.min    0.042682
34                   Production Qty Collect Result_Dam    0.035463
66                 Production Qty Collect Result_Fill1    0.035383
81                 Production Qty Collect Result_Fill2    0.034988
40               1st Pressure Collect Result_AutoClave    0.033295
65                       PalletID Collect Result_Fill1    0.031218
80                       PalletID Collect Result_Fill2    0.030506
33                         PalletID Collect Result_Dam    0.030122
42               2nd Pressure Collect Result_AutoClave    0.029428
46              Chamber Temp. Collect Result_AutoClave    0.028986
35                        Receip No Collect Result_Dam    0.02

### Xgboost

In [44]:
'''
# XGBClassifier 모델 생성
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss',enable_categorical=True)

# 탐색할 하이퍼파라미터 공간 정의
param_dist = {
    'n_estimators': Integer(100, 500),  # 트리의 개수
    'max_depth': Integer(3, 15),  # 트리의 최대 깊이
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),  # 학습률
    'subsample': Real(0.6, 1.0),  # 각 트리 학습에 사용할 데이터 샘플링 비율
    'colsample_bytree': Real(0.6, 1.0),  # 각 트리 학습에 사용할 특징 샘플링 비율
    'min_child_weight': Integer(1, 10),  # 과적합 방지를 위한 최소 가중치 합
    'gamma': Real(0, 5),  # 트리의 분할을 결정하는 최소 손실 감소 값
    'reg_alpha': Real(1e-8, 1.0, 'log-uniform'),  # L1 정규화 파라미터
    'reg_lambda': Real(1e-8, 1.0, 'log-uniform'),  # L2 정규화 파라미터
    'scale_pos_weight': Real(1e-2, 10.0, 'log-uniform'),  # 불균형 데이터 가중치 조정
}

# BayesSearchCV 객체 생성
bayes_search = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_dist,
    n_iter=50,  # 탐색할 하이퍼파라미터 조합 수
    cv=5,  # 교차 검증 폴드 수
    scoring='f1',  # 평가 지표
    random_state=42,  # 랜덤 시드 설정
    n_jobs=-1  # 사용 가능한 모든 CPU 사용
)

# BayesSearchCV 수행
bayes_search.fit(X_final, y_final)

# 최적의 하이퍼파라미터 출력
print("Best hyperparameters found: ", bayes_search.best_params_)

# 최적의 모델
best_model = bayes_search.best_estimator_
'''

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:04:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:04:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:04:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:04:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:04:39] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:06:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:06:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:06:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:06:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:06:40] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:16:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:18:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:18:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:18:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:18:30] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:26:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:26:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:26:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:26:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:26:33] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:31:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:32:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:32:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:32:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:32:05] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:38:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:38:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:38:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:38:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:38:20] WARNING: /workspace/sr

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:46:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:46:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:46:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:46:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:46:59] WARNING: /workspace/sr

Best hyperparameters found:  OrderedDict([('colsample_bytree', 1.0), ('gamma', 5.0), ('learning_rate', 0.01), ('max_depth', 15), ('min_child_weight', 10), ('n_estimators', 100), ('reg_alpha', 1e-08), ('reg_lambda', 9.457513359888871e-07), ('scale_pos_weight', 7.403475866301234), ('subsample', 0.9480267224163458)])


Best hyperparameters found:  OrderedDict([('colsample_bytree', 1.0), ('gamma', 5.0), ('learning_rate', 0.01), ('max_depth', 15), ('min_child_weight', 10), ('n_estimators', 100), ('reg_alpha', 1e-08), ('reg_lambda', 9.457513359888871e-07), ('scale_pos_weight', 7.403475866301234), ('subsample', 0.9480267224163458)])

In [48]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


f1_scores = []
precision_scores = []
recall_scores = []
feature_importances = []

# BayesSearchCV
best_params = {
    'n_estimators': 100,
    'learning_rate': 0.01,
    'max_depth': 15,
    'min_child_weight': 10,
    'subsample': 0.9480267224163458,
    'colsample_bytree': 1,
    'gamma': 5,
    'scale_pos_weight': 7.403475866301234,
    'use_label_encoder': False,  # 경고 방지
    #'eval_metric': 'logloss',
    'enable_categorical': True,  # categorical feature 지원
    'random_state': 42
    
}


for train_index, val_index in skf.split(X_final, y_final):
    X_train, X_val = X_final.iloc[train_index], X_final.iloc[val_index]
    y_train, y_val = y_final.iloc[train_index], y_final.iloc[val_index]


    xgbmodel = xgb.XGBClassifier(**best_params)
    xgbmodel.fit(X_train, y_train)

    # 검증 세트 예측 및 평가
    y_pred = xgbmodel.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)


    cm = confusion_matrix(y_val, y_pred)
    print('Confusion Matrix:')
    print(cm)

    print('F1 Score:', f1)
    print('Precision Score:', precision)
    print('Recall Score:', recall)
    print()


    feature_importances.append(xgbmodel.feature_importances_)


print("Average F1 Score:", np.mean(f1_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Recall:", np.mean(recall_scores))


mean_feature_importances = np.mean(feature_importances, axis=0)
features = X_final.columns
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': mean_feature_importances
}).sort_values(by='Importance', ascending=False)


print(importance_df.head(10))

/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:48:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
[[7539   93]
 [ 380   90]]
F1 Score: 0.27565084226646247
Precision Score: 0.4918032786885246
Recall Score: 0.19148936170212766



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:48:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
[[7557   74]
 [ 386   84]]
F1 Score: 0.267515923566879
Precision Score: 0.5316455696202531
Recall Score: 0.17872340425531916



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:49:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
[[7539   92]
 [ 373   97]]
F1 Score: 0.2943854324734446
Precision Score: 0.5132275132275133
Recall Score: 0.20638297872340425



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:49:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
[[7515  116]
 [ 393   77]]
F1 Score: 0.23227752639517346
Precision Score: 0.39896373056994816
Recall Score: 0.16382978723404254



/home/elicer/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:49:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix:
[[7562   69]
 [ 378   92]]
F1 Score: 0.29160063391442154
Precision Score: 0.5714285714285714
Recall Score: 0.19574468085106383

Average F1 Score: 0.2722860717232762
Average Precision: 0.5014137327069621
Average Recall: 0.18723404255319148
                                          Feature  Importance
1                                   Workorder_Dam    0.077051
122                            production_qty_std    0.039478
86                            AutoClave-Fill2.min    0.027325
116                       head_normal_y_fill1_std    0.012980
0                                Model.Suffix_Dam    0.012647
52   Dispense Volume(Stage1) Collect Result_Fill1    0.011986
2                                Collect Date_Dam    0.011793
53   Dispense Volume(Stage2) Collect Result_Fill1    0.011247
69                CURE SPEED Collect Result_Fill2    0.011090
39                    WorkMode Collect Result_Dam    0.010825


In [49]:
print(importance_df.head(30))

                                               Feature  Importance
1                                        Workorder_Dam    0.077051
122                                 production_qty_std    0.039478
86                                 AutoClave-Fill2.min    0.027325
116                            head_normal_y_fill1_std    0.012980
0                                     Model.Suffix_Dam    0.012647
52        Dispense Volume(Stage1) Collect Result_Fill1    0.011986
2                                     Collect Date_Dam    0.011793
53        Dispense Volume(Stage2) Collect Result_Fill1    0.011247
69                     CURE SPEED Collect Result_Fill2    0.011090
39                         WorkMode Collect Result_Dam    0.010825
83                       WorkMode Collect Result_Fill2    0.010747
37                      THICKNESS 2 Collect Result_Dam    0.010660
54        Dispense Volume(Stage3) Collect Result_Fill1    0.010576
111                         dispensed_volume_fill1_std    0.01

In [50]:
'''
import shap


# SHAP Explainer 생성
explainer = shap.Explainer(xgbmodel)

# SHAP 값을 계산
shap_values = explainer(X_train)

# 변수 중요도 시각화
shap.summary_plot(shap_values, X_train)
'''

'\nimport shap\n\n\n# SHAP Explainer 생성\nexplainer = shap.Explainer(xgbmodel)\n\n# SHAP 값을 계산\nshap_values = explainer(X_train)\n\n# 변수 중요도 시각화\nshap.summary_plot(shap_values, X_train)\n'

In [51]:
#shap_values

## 4. 제출하기


### 테스트 데이터 예측 (ensemble)


테스트 데이터 불러오기


In [52]:
df_test_final.head()

,Set ID,Model.Suffix_Dam,Workorder_Dam,Collect Date_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,...,production_qty_std,pressure_autoclave_std,pressure_time_autoclave_std,THICKNESS 2+3 Collect Result_Dam,Stage1 Circle1~4 Distance Speed Collect Result_Dam,Stage1 Line1~4 Distance Speed Collect Result_Dam,Stage2 Circle1~4 Distance Speed Collect Result_Dam,Stage2 Line1~4 Distance Speed Collect Result_Dam,Stage3 Circle1~4 Distance Speed Collect Result_Dam,Stage3 Line1~4 Distance Speed Collect Result_Dam
0,0,0,240,3200,70,10,17.0,4.9,17.0,1.19,...,0.0,0.105368,69.282032,0.000,5000.00,5000.0,9000.0,9000.0,5000.0,5000.0
1,1,0,509,6382,70,16,14.2,8.3,14.2,0.99,...,0.0,0.166206,120.000000,-0.273,4500.25,6000.0,5625.0,5500.0,5875.0,6000.0
2,2,0,128,2053,70,10,9.7,4.9,9.7,0.67,...,0.0,0.110851,69.282032,0.000,9000.00,9000.0,9000.0,9000.0,9000.0,9000.0
3,3,0,306,4332,70,10,21.3,10.6,21.3,1.49,...,0.0,0.114605,79.751698,0.000,3000.25,4000.0,4000.0,4000.0,4000.0,4000.0
4,4,0,415,5427,70,16,13.2,7.5,13.2,0.92,...,0.0,0.111168,120.000000,0.000,4875.25,6500.0,6500.0,6000.0,6500.0,6500.0


In [53]:
df_test_x = df_test_final.drop('Set ID',axis=1)



rf_pred = rfmodel.predict(df_test_x)
xgb_pred = xgbmodel.predict(df_test_x)



ensemble_proba = rf_pred * 0.5 + xgb_pred * 0.5
ensemble_pred = (ensemble_proba >= 0.5).astype(int) # 둘 중 하나라도 1이면 1로 판정하게끔


# 결과 확인
print(pd.DataFrame(ensemble_pred).value_counts())

test_pred = ensemble_pred


0    16879
1      482
Name: count, dtype: int64


In [54]:
pd.DataFrame([rf_pred,xgb_pred]).T.value_counts()

0  1
0  0    16879
1  1      186
0  1      165
1  0      131
Name: count, dtype: int64

In [55]:
pd.Series(test_pred).value_counts()

0    16879
1      482
Name: count, dtype: int64

In [56]:
# 'test_pred' contains the numerical predictions (0 or 1)
result_labels = ['Normal' if pred == 0 else 'AbNormal' for pred in test_pred]

# Print the results with labels
pd.Series(result_labels).value_counts()

Normal      16879
AbNormal      482
Name: count, dtype: int64

### 제출 파일 작성


In [57]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = result_labels

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
